In [22]:
import snowflake.connector
import psycopg2
import csv
from io import StringIO
import pandas as pd

In [23]:
snowflake_connection = snowflake.connector.connect(
         
        )

In [24]:
snowflake_cursor = snowflake_connection.cursor()


# Check Snowflake connection
try:
    snowflake_connection
    print("Connection to Snowflake successful!")
    
except Exception as e:
    print(f"Error connecting to Snowflake: {str(e)}")



Connection to Snowflake successful!


In [25]:
try:
    # Connect to PostgreSQL
    pg_conn = psycopg2.connect(
        dbname='testdb',
        user='postgres',
        password='password',
        host='localhost',
        port='5432'  # Default PostgreSQL port
    )

    # Create a cursor object using the connection
    pg_cursor = pg_conn.cursor()
    print("Connection to PostgreSQL successful.")
except psycopg2.Error as e:
    print("Error connecting to PostgreSQL:", e)

Connection to PostgreSQL successful.


In [26]:
pg_cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public' AND table_type = 'BASE TABLE'")
tables = pg_cursor.fetchall()

In [31]:
for table in tables:
    table_name = table[0]
    # Read data from the table into a pandas DataFrame
    df = pd.read_sql(f'SELECT * FROM {table_name}', pg_conn)
    # Export DataFrame to CSV
    df.to_csv(f'staging/{table_name}.csv', index=False)
    print(f"Table '{table_name}' exported to CSV successfully.")

Table 'tbl_skills' exported to CSV successfully.
Table 'tbl_certificates' exported to CSV successfully.
Table 'tbl_user_project' exported to CSV successfully.
Table 'tbl_user' exported to CSV successfully.
Table 'tbl_approver_user' exported to CSV successfully.
Table 'tbl_user_certificate' exported to CSV successfully.
Table 'tbl_user_skill' exported to CSV successfully.


C:\Users\SwarnadeepPramanik\AppData\Local\Temp\ipykernel_11824\4047154470.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {table_name}', pg_conn)


In [33]:
pg_conn.close()

# ingestion

In [35]:
import os

In [38]:
staging_folder = 'staging/'  # Change this to your actual staging folder path

# Get a list of all CSV files in the staging folder
csv_files = [file for file in os.listdir(staging_folder) if file.endswith('.csv')]

In [41]:
os.path.splitext(csv_files[0])[0] 

'tbl_approver_user'

In [48]:
temp = pd.read_csv(os.path.join(staging_folder, csv_files[0]))

In [52]:
for csv_file in csv_files:
    table_name = os.path.splitext(csv_file)[0]  # Use CSV file name as table name
    
    # Read CSV file into DataFrame
    df = pd.read_csv(os.path.join(staging_folder, csv_file))
    
    # Create table in Snowflake
    create_table_query = f"CREATE TABLE {table_name} ("
    for column in df.columns:
        create_table_query += f"{column} VARCHAR,"
    create_table_query = create_table_query[:-1]  # Remove trailing comma
    create_table_query += ")"

    # print(create_table_query)
    # break
    snowflake_cursor.execute(create_table_query)
    print(f"Table '{table_name}' created in Snowflake.")
    
    # Insert data into Snowflake table
    snowflake_cursor.executemany(f"INSERT INTO {table_name} VALUES ({','.join(['%s']*len(df.columns))})", df.values.tolist())
    print(f"Data inserted into table '{table_name}' in Snowflake.")
    
# Commit changes
snowflake_connection.commit()

Table 'tbl_approver_user' created in Snowflake.
Data inserted into table 'tbl_approver_user' in Snowflake.
Table 'tbl_certificates' created in Snowflake.
Data inserted into table 'tbl_certificates' in Snowflake.
Table 'tbl_skills' created in Snowflake.
Data inserted into table 'tbl_skills' in Snowflake.
Table 'tbl_user' created in Snowflake.
Data inserted into table 'tbl_user' in Snowflake.
Table 'tbl_user_certificate' created in Snowflake.
Data inserted into table 'tbl_user_certificate' in Snowflake.
Table 'tbl_user_project' created in Snowflake.
Data inserted into table 'tbl_user_project' in Snowflake.
Table 'tbl_user_skill' created in Snowflake.
Data inserted into table 'tbl_user_skill' in Snowflake.


In [53]:
# Close cursor and connection
snowflake_connection.close()
pg_conn.close()
